# Create a visualization of ATRACKCS performance


In [20]:
import numpy as np 
import pandas as pd
import xarray as xr
import glob
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.wkt import loads
import os
from matplotlib.cm import get_cmap
import matplotlib.patches as mpatches

In [21]:
# === Load Data ===
# Path to ATRACKCS CSV file
csv_file_path = '../tests/Results/resume_DeepConvection_2016-08-01 0_2016-08-04.csv'
atracks_resume = pd.read_csv(csv_file_path, sep=',')

# Load brightness temperature data (Tb)
wa_nc = xr.open_mfdataset('../tests/WA/tb/*.nc4', engine='netcdf4')
tb = wa_nc['Tb']
tb = tb.resample(time="1h").nearest(tolerance="1h")


# Convert geometry column to shapely objects
atracks_resume['geometry'] = atracks_resume['geometry'].apply(loads)
atrackcs_gdf = gpd.GeoDataFrame(atracks_resume, geometry='geometry')
atrackcs_gdf.set_crs("EPSG:4326", inplace=True)

# Convert 'time' column to datetime
atrackcs_gdf['time'] = pd.to_datetime(atrackcs_gdf['time'])

# === Domain Extent ===
lonW, lonE = wa_nc.lon.values[0], wa_nc.lon.values[-1]
latS, latN = wa_nc.lat.values[0], wa_nc.lat.values[-1]

# === Colormap Setup ===
unique_belong_ids = sorted(atrackcs_gdf['track_id'].unique())
n_ids = len(unique_belong_ids)
# Use a continuous colormap (e.g., tab20, tab20c, viridis, etc.)
cmap = plt.colormaps['tab20']  # or plt.cm.get_cmap('tab20')
# Map each ID to a unique color by sampling evenly across the colormap
color_mapping = {
    belong: cmap(i / n_ids) for i, belong in enumerate(unique_belong_ids)
}

# === Output Folder ===
output_folder = "../tests/Results/Figures/"
os.makedirs(output_folder, exist_ok=True)




In [22]:
# === Plotting Loop ===
# In this example, I'm just plotting 2 days for demonstration
# You can adjust the range as needed

fig = plt.figure(figsize=(10, 6))

for i, time in enumerate(tb['time'].values[:48]):  # First 48 hours
    timestamp = pd.Timestamp(time)
    
    # Match time exactly or approximately
    time_gdf = atrackcs_gdf[atrackcs_gdf['time'] == timestamp]

    fig.clf()
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())

    # Plot brightness temperature
    tb.sel(time=time).plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greys', 
                           levels=np.arange(180, 300, 1),
                           cbar_kwargs={'shrink': 0.6, 'aspect': 20, 'pad': 0.01, 'label': 'Tb (K)'})
    masked_t = tb.sel(time=time).where(tb.sel(time=time) <= 241)
    masked_t.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), cmap='Spectral', levels = np.arange(180,241,1),
                           cbar_kwargs={'shrink': 0.6, 'aspect': 20, 'pad': 0.01, 'label': 'Tb_241 (K)'})
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='white')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='gray')

    # Plot polygons
    for _, row in time_gdf.iterrows():
        color = color_mapping.get(row['track_id'], 'black')
        ax.add_geometries([row['geometry']], crs=ccrs.PlateCarree(),
                          edgecolor=color, facecolor='none', linewidth=2)

    ax.set_title(f"Time: {timestamp} UTC")

    # Save figure
    fig_path = os.path.join(output_folder, f"frame_{i:04d}.png")
    plt.savefig(fig_path, dpi=150, bbox_inches='tight')
    #plt.show()

plt.close(fig)
print("Frames saved in:", output_folder)

Frames saved in: ../tests/Results/Figures/


Now let's create a GIF file.

In [23]:
from PIL import Image
import glob

# Get PNG files (ensure they are in the right order)
png_files = sorted(glob.glob(f"{output_folder}*.png"))

# Open all frames
frames = [Image.open(f).convert("RGB") for f in png_files]

# Save as a multi-frame (animated) TIFF
frames[0].save(
    f"{output_folder}/ATRACKCS_animation.gif",
    save_all=True,
    append_images=frames[1:],
    duration=250,   # time per frame in milliseconds
    loop=0          # 0 = infinite loop
)